In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

# READING FILES

### EDO DE CUENTA

In [2]:
all_mov = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\CONCILIACIONES\2024\01. ENERO\EDO CTA BNTE\*.txt', recursive=False)
all_mov

['C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2024\\01. ENERO\\EDO CTA BNTE\\BNTE 2024.01.02 PT1.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2024\\01. ENERO\\EDO CTA BNTE\\BNTE 2024.01.02 PT10.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2024\\01. ENERO\\EDO CTA BNTE\\BNTE 2024.01.02 PT11.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2024\\01. ENERO\\EDO CTA BNTE\\BNTE 2024.01.02 PT12.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2024\\01. ENERO\\EDO CTA BNTE\\BNTE 2024.01.02 PT13.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2024\\01. ENERO\\EDO CTA BNTE\\BNTE 2024.01.02 PT14.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2024\\01. ENERO\\EDO CTA BNTE\\BNTE 2024.01.02 PT15.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2024\\01. ENERO\\EDO CTA BNTE\\BNTE 2024.01.02 PT16.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2024\\

In [3]:
edo_cta = pd.concat([pd.read_csv(f,sep='|',dtype={'Cuenta': object,'Cod. Transac':object,'Referencia':object})
                     .assign(file_name=os.path.basename(f)) for f in all_mov])
edo_cta.reset_index(inplace=True)
del edo_cta['index']
edo_cta.sort_values(by=['Movimiento'],inplace=True)
edo_cta['Cuenta'] = "'"+edo_cta['Cuenta']
edo_cta['Descripción'].fillna('SIN DESCRIPCION',inplace=True)
edo_cta.head()

,Cuenta,Fecha De Operación,Fecha,Referencia,Descripción,Cod. Transac,Sucursal,Depósitos,Retiros,Saldo,Movimiento,Descripción Detallada,Cheque,file_name
0,'0239650517,30/12/2023,02/01/2024,0300001,COMPRA ORDEN DE PAGO SPEI,511,4264,$0.00,$934.28,"$501,321,740.73",41507057,"CTA/CLABE:002760701768361110 ,CB SPEI, BCO: 00...",NaN,BNTE 2024.01.02 PT1.txt
1,'0239650517,30/12/2023,02/01/2024,0363373,COMPRA ORDEN DE PAGO SPEI,511,4264,$0.00,$635.32,"$501,321,105.41",41507058,"CTA/CLABE:014700230013272347 ,CB SPEI, BCO: 01...",NaN,BNTE 2024.01.02 PT1.txt
2,'0239650517,30/12/2023,02/01/2024,0331988,COMPRA ORDEN DE PAGO SPEI,511,4264,$0.00,"$2,108.14","$501,318,997.27",41507059,"CTA/CLABE:012180004693203856 ,CB SPEI, BCO: 01...",NaN,BNTE 2024.01.02 PT1.txt
3,'0239650517,30/12/2023,02/01/2024,0300006,COMPRA ORDEN DE PAGO SPEI,511,4264,$0.00,$527.03,"$501,318,470.24",41507060,"CTA/CLABE:044010256029105948 ,CB SPEI, BCO: 04...",NaN,BNTE 2024.01.02 PT1.txt
4,'0239650517,30/12/2023,02/01/2024,0363378,COMPRA ORDEN DE PAGO SPEI,511,4264,$0.00,"$2,801.91","$501,315,668.33",41507061,"CTA/CLABE:021694064625716414 ,CB SPEI, BCO: 02...",NaN,BNTE 2024.01.02 PT1.txt


In [4]:
edo_cta.shape

(1230196, 14)

In [5]:
split_fecha_operacion = edo_cta["Fecha De Operación"].str.split("/",n=3,expand=True)
split_fecha_operacion.head()

,0,1,2
0,30,12,2023
1,30,12,2023
2,30,12,2023
3,30,12,2023
4,30,12,2023


In [6]:
edo_cta["Day_operacion"] = split_fecha_operacion[0] 
edo_cta["Month_operacion"] = split_fecha_operacion[1]
edo_cta['Year_operacion'] = split_fecha_operacion[2]

In [7]:
split_fecha_banco = edo_cta["Fecha"].str.split("/",n=3,expand=True)
split_fecha_banco.head()

,0,1,2
0,02,01,2024
1,02,01,2024
2,02,01,2024
3,02,01,2024
4,02,01,2024


In [8]:
edo_cta["Day_banco"] = split_fecha_banco[0] 
edo_cta["Month_banco"] = split_fecha_banco[1]
edo_cta['Year_banco'] = split_fecha_banco[2]

In [9]:
edo_cta['Fecha_operacion'] = edo_cta['Year_operacion']+'-'+edo_cta['Month_operacion']+'-'+edo_cta['Day_operacion']
edo_cta['Fecha_banco'] = edo_cta['Year_banco']+'-'+edo_cta['Month_banco']+'-'+edo_cta['Day_banco']

In [10]:
edo_cta['Fecha_operacion'] = pd.to_datetime(edo_cta['Fecha_operacion'])
edo_cta['Fecha_banco'] = pd.to_datetime(edo_cta['Fecha_banco'])

In [11]:
edo_cta.loc[edo_cta['Descripción'].str.contains('08343798'), "Tipo"] = 'XPAY'
edo_cta.loc[edo_cta['Descripción'].str.contains('08380402'), "Tipo"] = 'MIT-3DS'
edo_cta.loc[(~edo_cta['Descripción'].str.contains('08343798')) & (~edo_cta['Descripción'].str.contains('08380402')), "Tipo"] = 'REGULAR TXN'


In [12]:
edo_cta.Depósitos = [x.strip('$') for x in edo_cta.Depósitos]
edo_cta.Retiros = [x.strip('$') for x in edo_cta.Retiros]
edo_cta.Saldo = [x.strip('$') for x in edo_cta.Saldo]


In [13]:
edo_cta['Depósitos'].replace(',','', regex=True, inplace=True)
edo_cta['Retiros'].replace(',','', regex=True, inplace=True)
edo_cta['Saldo'].replace(',','', regex=True, inplace=True)

In [15]:
edo_cta['Depósitos'] = edo_cta['Depósitos'].astype(float)
edo_cta['Retiros'] = edo_cta['Retiros'].astype(float)
#edo_cta['Saldo'] = edo_cta['Saldo'].astype(float)

In [16]:
edo_cta.loc[edo_cta['Cod. Transac'] == '060', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[edo_cta['Cod. Transac'] == '061', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[edo_cta['Cod. Transac'] == '062', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[edo_cta['Cod. Transac'] == '600', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[edo_cta['Cod. Transac'] == '601', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[edo_cta['Cod. Transac'] == '602', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[(edo_cta['Cod. Transac'] == '602') & (edo_cta['Descripción Detallada'].str.contains('CONTRACARGO')), 'Afiliacion_1'] = edo_cta['Descripción'].str[-8:]
edo_cta['len']= edo_cta['Afiliacion_1'].str.len() 
edo_cta.loc[edo_cta['len'] == 9, 'Afiliacion_final'] = edo_cta['Afiliacion_1'].str[-8:-1]
edo_cta.loc[edo_cta['len'] == 8, 'Afiliacion_final'] = edo_cta['Afiliacion_1'].str[-7:]
edo_cta.loc[edo_cta['len'] == 9, 'Tipo_operacion'] = edo_cta['Afiliacion_1'].str[-1:]
edo_cta.head()

,Cuenta,Fecha De Operación,Fecha,Referencia,Descripción,Cod. Transac,Sucursal,Depósitos,Retiros,Saldo,...,Day_banco,Month_banco,Year_banco,Fecha_operacion,Fecha_banco,Tipo,Afiliacion_1,len,Afiliacion_final,Tipo_operacion
0,'0239650517,30/12/2023,02/01/2024,0300001,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,934.28,501321740.73,...,02,01,2024,2023-12-30,2024-01-02,REGULAR TXN,NaN,NaN,NaN,NaN
1,'0239650517,30/12/2023,02/01/2024,0363373,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,635.32,501321105.41,...,02,01,2024,2023-12-30,2024-01-02,REGULAR TXN,NaN,NaN,NaN,NaN
2,'0239650517,30/12/2023,02/01/2024,0331988,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,"2,108.14",501318997.27,...,02,01,2024,2023-12-30,2024-01-02,REGULAR TXN,NaN,NaN,NaN,NaN
3,'0239650517,30/12/2023,02/01/2024,0300006,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,527.03,501318470.24,...,02,01,2024,2023-12-30,2024-01-02,REGULAR TXN,NaN,NaN,NaN,NaN
4,'0239650517,30/12/2023,02/01/2024,0363378,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,"2,801.91",501315668.33,...,02,01,2024,2023-12-30,2024-01-02,REGULAR TXN,NaN,NaN,NaN,NaN


In [17]:
edo_cta.shape

(1230196, 27)

In [18]:
all_edo = edo_cta[['Cuenta','Fecha_operacion','Fecha_banco','Referencia','Descripción','Cod. Transac','Sucursal','Depósitos'
                   ,'Retiros','Saldo','Movimiento','Descripción Detallada','Cheque','Tipo','Afiliacion_1','Afiliacion_final'
                   ,'Tipo_operacion']]
all_edo.fillna('BLANK',inplace=True)
all_edo.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_20112\605945623.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'BLANK' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  all_edo.fillna('BLANK',inplace=True)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_20112\605945623.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_edo.fillna('BLANK',inplace=True)


,Cuenta,Fecha_operacion,Fecha_banco,Referencia,Descripción,Cod. Transac,Sucursal,Depósitos,Retiros,Saldo,Movimiento,Descripción Detallada,Cheque,Tipo,Afiliacion_1,Afiliacion_final,Tipo_operacion
0,'0239650517,2023-12-30,2024-01-02,0300001,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,934.28,501321740.73,41507057,"CTA/CLABE:002760701768361110 ,CB SPEI, BCO: 00...",BLANK,REGULAR TXN,BLANK,BLANK,BLANK
1,'0239650517,2023-12-30,2024-01-02,0363373,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,635.32,501321105.41,41507058,"CTA/CLABE:014700230013272347 ,CB SPEI, BCO: 01...",BLANK,REGULAR TXN,BLANK,BLANK,BLANK
2,'0239650517,2023-12-30,2024-01-02,0331988,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,"2,108.14",501318997.27,41507059,"CTA/CLABE:012180004693203856 ,CB SPEI, BCO: 01...",BLANK,REGULAR TXN,BLANK,BLANK,BLANK
3,'0239650517,2023-12-30,2024-01-02,0300006,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,527.03,501318470.24,41507060,"CTA/CLABE:044010256029105948 ,CB SPEI, BCO: 04...",BLANK,REGULAR TXN,BLANK,BLANK,BLANK
4,'0239650517,2023-12-30,2024-01-02,0363378,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,"2,801.91",501315668.33,41507061,"CTA/CLABE:021694064625716414 ,CB SPEI, BCO: 02...",BLANK,REGULAR TXN,BLANK,BLANK,BLANK


In [19]:
all_edo.shape

(1230196, 17)

In [20]:
amex_edo_cta = all_edo[(all_edo['Descripción Detallada'].str.contains('AMERICAN EXPRESS'))]
amex_edo_cta.head()


,Cuenta,Fecha_operacion,Fecha_banco,Referencia,Descripción,Cod. Transac,Sucursal,Depósitos,Retiros,Saldo,Movimiento,Descripción Detallada,Cheque,Tipo,Afiliacion_1,Afiliacion_final,Tipo_operacion
274079,'0239650517,2024-01-02,2024-01-02,0000000000,20240102400140HDH0000461811370,003,5663,202.21,0.00,324580507.97,41801136,"SPEI RECIBIDO, BCO:0014 SANTANDER HR...",BLANK,REGULAR TXN,BLANK,BLANK,BLANK
274080,'0239650517,2024-01-02,2024-01-02,0000000000,18CCA2D42F9B1BEC,003,5663,"73,625,189.97",0.00,398205697.94,41801137,"SPEI RECIBIDO, BCO:0002 BANAMEX HR...",BLANK,REGULAR TXN,BLANK,BLANK,BLANK
295819,'0239650517,2024-01-03,2024-01-03,0000000000,18CCF5A3D2F1CABF,003,5663,"16,440,936.84",0.00,362422549.47,41802876,"SPEI RECIBIDO, BCO:0002 BANAMEX HR...",BLANK,REGULAR TXN,BLANK,BLANK,BLANK
297566,'0239650517,2024-01-04,2024-01-04,0000000000,18CD481DEE4BE150,003,5663,"18,832,457.55",0.00,378187833.66,41804623,"SPEI RECIBIDO, BCO:0002 BANAMEX HR...",BLANK,REGULAR TXN,BLANK,BLANK,BLANK
299830,'0239650517,2024-01-05,2024-01-05,0000000000,18CD99E68322E1EB,003,5663,"19,498,903.35",0.00,384654078.05,41806887,"SPEI RECIBIDO, BCO:0002 BANAMEX HR...",BLANK,REGULAR TXN,BLANK,BLANK,BLANK


In [21]:
amex_edo_cta.shape

(24, 17)

In [22]:
amex_edo_cta_pivot = amex_edo_cta.groupby(['Fecha_operacion','Fecha_banco']).agg({'Depósitos':sum,'Retiros':sum}).reset_index()
amex_edo_cta_pivot.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_20112\3039474254.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  amex_edo_cta_pivot = amex_edo_cta.groupby(['Fecha_operacion','Fecha_banco']).agg({'Depósitos':sum,'Retiros':sum}).reset_index()


,Fecha_operacion,Fecha_banco,Depósitos,Retiros
0,2024-01-02,2024-01-02,"73,625,392.18",0.00
1,2024-01-03,2024-01-03,"16,440,936.84",0.00
2,2024-01-04,2024-01-04,"18,832,457.55",0.00
3,2024-01-05,2024-01-05,"19,498,903.35",0.00
4,2024-01-08,2024-01-08,"53,984,833.70",0.00


In [23]:
amex_edo_cta_pivot.shape

(22, 4)

In [24]:
writer = pd.ExcelWriter('EDO CTA ENERO 2024.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
amex_edo_cta.to_excel(writer, sheet_name='DETALLE',index=False)
amex_edo_cta_pivot.to_excel(writer, sheet_name='BY DATE',index=False)
writer.close()